# Asymmetric Bounded-Confidence Agent-Based Model Demo

This notebook demonstrates the **Asymmetric Bounded-Confidence Model** for opinion dynamics simulation. The method modulates confidence bounds based on perception asymmetry (alpha) - agents perceive their ingroup as more diverse than outgroups, discounting outgroup signals by a factor of (1-alpha).

**Hypothesis**: Higher perception asymmetry (alpha) leads to more stable opinion bubbles because agents discount outgroup signals, preventing cluster merging.

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

## 1. Load Demo Data

We load pre-computed simulation results from `demo_data.json`. Each example contains baseline (alpha=0) and method (alpha>0) results for comparison.

In [ ]:
# Load data from JSON file
with open('demo_data.json', 'r') as f:
    data = json.load(f)

examples = data['examples']
print(f"Loaded {len(examples)} examples")
print(f"\nFirst example keys: {list(examples[0].keys())}")

## 2. Parse and Organize Results

Extract baseline and method predictions from the JSON data into a structured DataFrame.

In [ ]:
# Parse results into DataFrame
rows = []
for ex in examples:
    baseline = json.loads(ex['predict_baseline'])
    method = json.loads(ex['predict_method'])
    
    rows.append({
        'seed': ex['context']['seed'],
        'method_alpha': ex['context']['method_alpha'],
        # Baseline metrics
        'baseline_clusters': baseline['final_cluster_count'],
        'baseline_lifetime': baseline['cluster_lifetime_mean'],
        'baseline_variance': baseline['final_variance'],
        'baseline_conv_time': baseline['convergence_time'],
        # Method metrics
        'method_clusters': method['final_cluster_count'],
        'method_lifetime': method['cluster_lifetime_mean'],
        'method_variance': method['final_variance'],
        'method_conv_time': method['convergence_time'],
    })

df = pd.DataFrame(rows)
print(f"DataFrame shape: {df.shape}")
df.head(10)

## 3. Visualize: Alpha vs Cluster Lifetime

The key hypothesis is that higher alpha values lead to longer cluster lifetimes (more stable opinion bubbles).

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Alpha vs Cluster Lifetime
ax1 = axes[0]
for seed in df['seed'].unique():
    subset = df[df['seed'] == seed]
    ax1.scatter(subset['method_alpha'], subset['method_lifetime'], 
                alpha=0.7, label=f'Seed {seed}')

# Add trend line
z = np.polyfit(df['method_alpha'], df['method_lifetime'], 1)
p = np.poly1d(z)
alpha_range = np.linspace(df['method_alpha'].min(), df['method_alpha'].max(), 100)
ax1.plot(alpha_range, p(alpha_range), 'k--', linewidth=2, label='Trend')

ax1.set_xlabel('Perception Asymmetry (alpha)', fontsize=12)
ax1.set_ylabel('Cluster Lifetime', fontsize=12)
ax1.set_title('Effect of Perception Asymmetry on Opinion Bubble Stability', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Baseline vs Method comparison (bar chart)
ax2 = axes[1]
alpha_groups = df.groupby('method_alpha').agg({
    'baseline_lifetime': 'mean',
    'method_lifetime': 'mean'
}).reset_index()

x = np.arange(len(alpha_groups))
width = 0.35
ax2.bar(x - width/2, alpha_groups['baseline_lifetime'], width, 
        label='Baseline (alpha=0)', color='steelblue', alpha=0.8)
ax2.bar(x + width/2, alpha_groups['method_lifetime'], width, 
        label='Method (alpha>0)', color='coral', alpha=0.8)

ax2.set_xlabel('Method Alpha Value', fontsize=12)
ax2.set_ylabel('Mean Cluster Lifetime', fontsize=12)
ax2.set_title('Baseline vs Method: Cluster Stability Comparison', fontsize=14)
ax2.set_xticks(x)
ax2.set_xticklabels([f'{a:.2f}' for a in alpha_groups['method_alpha']])
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 4. Statistical Analysis

Compute correlation between alpha and cluster lifetime, and perform a t-test comparing baseline vs method.

In [ ]:
# Correlation analysis
r, p_corr = stats.pearsonr(df['method_alpha'], df['method_lifetime'])
print("=" * 50)
print("CORRELATION ANALYSIS")
print("=" * 50)
print(f"Pearson correlation (alpha vs lifetime): r = {r:.4f}")
print(f"P-value: {p_corr:.6f}")
print(f"Interpretation: {'Significant' if p_corr < 0.05 else 'Not significant'} at alpha=0.05")

# T-test: baseline vs method lifetime
t_stat, p_ttest = stats.ttest_rel(df['baseline_lifetime'], df['method_lifetime'])
print("\n" + "=" * 50)
print("PAIRED T-TEST: BASELINE vs METHOD")
print("=" * 50)
print(f"Mean baseline lifetime: {df['baseline_lifetime'].mean():.2f}")
print(f"Mean method lifetime: {df['method_lifetime'].mean():.2f}")
print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_ttest:.6f}")
print(f"Interpretation: {'Significant difference' if p_ttest < 0.05 else 'No significant difference'} at alpha=0.05")

## 5. Summary Statistics by Alpha

In [ ]:
# Group by alpha and compute stats
summary = df.groupby('method_alpha').agg({
    'method_lifetime': ['mean', 'std'],
    'method_clusters': 'mean',
    'method_conv_time': 'mean'
}).round(2)

summary.columns = ['Lifetime Mean', 'Lifetime Std', 'Clusters Mean', 'Conv. Time Mean']
print("Summary Statistics by Alpha Value:")
summary

## 6. Conclusion

The results demonstrate that increasing perception asymmetry (alpha) leads to:
1. **More stable opinion clusters** (longer lifetime)
2. **Persistent opinion bubbles** that resist merging

This supports the hypothesis that when agents discount outgroup signals (higher alpha), opinion bubbles become more stable and resistant to change.

In [ ]:
# Final summary
context = examples[0]['context']
print("=" * 60)
print("EXPERIMENT CONFIGURATION")
print("=" * 60)
print(f"Network type: {context['network_type']}")
print(f"Number of agents: {context['n_agents']}")
print(f"Max timesteps: {context['max_timesteps']}")
print(f"Dataset: {examples[0]['dataset']}")
print(f"\nMethod: {examples[0]['method'][:200]}...")